Get the average temporal correlation values of all the spontaneous activities in one exp for 2 cells

In [64]:
# ec_id = 511510650
eid = 501836392 #501886692 three_session_B 501836392 three_session_A 501717543 three_session_C
c1 = 517419703 
c2 = 517419533 

Get exps for these cells

In [55]:
# List of stimulus info = https://github.com/AllenInstitute/AllenSDK/blob/63f7bb8a826a05cab49723904aa8b89c1e08687d/allensdk/brain_observatory/stimulus_info.py
cell_exps = boc.get_ophys_experiments(cell_specimen_ids=[c1],
                                     stimuli=[stim_info.SPONTANEOUS_ACTIVITY])
                                     #experiment_container_ids=[511510650])
#cell_exps

In [65]:
import os
drive_path = '/media/stephen/Brain20191/allen-brain-observatory/visual-coding-2p/'
if not os.path.isdir(drive_path):
    drive_path = '/media/alihamodi/Brain2019/allen-brain-observatory/visual-coding-2p/'
    
import warnings;
warnings.simplefilter('ignore');
import os
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
from utils.data_wrappers import *
from utils.polardata import *
from utils.polarplot import *
manifest_file = os.path.join(drive_path,'manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)

In [70]:
from scipy.stats import pearsonr

def avg_temp_corr_one_exp(boc, eid, c1, c2):
    """For one experiment, get the spontaneous presentations, for each one get temporal correlation of
    the two cells, then average all these temporal correlations.
    I checked that sesh A and B just have 1 spontaenous preso, session C have 2.
    So, only the big grey chunks are counted. The small ones aren't.
    """
    data_set = boc.get_ophys_experiment_data(eid)

    try: 
        cidxs = data_set.get_cell_specimen_indices([c1, c2])
    except Exception as inst:
        return None

    events = boc.get_ophys_experiment_events(ophys_experiment_id=eid)
    cidx1 = cidxs[0]
    cidx2 = cidxs[1]
    events1 = events[cidx1,:]
    events2 = events[cidx2,:]

    stim_table = data_set.get_stimulus_table('spontaneous') 

    # Each item is a temporal correlation of a single spontaneous presentation
    temp_corr_lists = []
    for i in range(len(stim_table)):
        start = stim_table.start[i]
        end = stim_table.end[i]
        ts1 = events1[start:end]
        ts2 = events2[start:end]
        temp_corr, p_value = pearsonr(ts1, ts2)
        temp_corr_lists.append(temp_corr)
    if len(temp_corr_lists) == 0:
        return None
    return np.mean(temp_corr_lists)

avg_temp_corr_one_exp(boc, eid, c1, c2)

-0.0019509815338686607